<a href="https://colab.research.google.com/github/yl540016/PortfolioProjects/blob/main/CDCdatawrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Which gender is more likely to get Covid in NY?**

In [1]:
import os
from glob import glob
os.getcwd() 
  # current working directory

'/content'

In [2]:
import pandas as pd
df_cdc = pd.read_csv('./covid19_cdc_datasets.csv')

### Exploring the dataframe

In [3]:
df_cdc.head()

,case_month,res_state,res_county,age_group,sex,race
0,2020-01,NY,ONEIDA,18 to 49 years,Male,NaN
1,2020-01,NY,ONEIDA,18 to 49 years,Male,NaN
2,2020-01,NY,ONEIDA,18 to 49 years,Male,NaN
3,2020-01,NY,ONEIDA,18 to 49 years,Male,NaN
4,2020-01,NY,ONEIDA,18 to 49 years,Male,NaN


In [5]:
# print the number of records and the number of columns (shape of pandas.DataFrame).
num_row = df_cdc.shape[0]
num_col = df_cdc.shape[1]

num_row, num_col

(50000, 6)

In [6]:
#print the list of column names
df_cdc.columns


Index(['case_month', 'res_state', 'res_county', 'age_group', 'sex', 'race'], dtype='object')

In [8]:
# print unique values and their counts in the sex col
df_cdc['sex'].unique()


array(['Male', nan, 'Female', 'Unknown'], dtype=object)

In [9]:
df_cdc['sex'].value_counts()

Male       27017
Female     21047
Unknown      112
Name: sex, dtype: int64

In [10]:
df_cdc['sex'].value_counts().sum()

48176

In [11]:
df_cdc['sex'].isna().sum()

1824

In [12]:
df_cdc['res_state'].value_counts()

NY    50000
Name: res_state, dtype: int64

In [13]:
df_cdc['sex'].unique()

array(['Male', nan, 'Female', 'Unknown'], dtype=object)

In [14]:
df_cdc['sex'].value_counts().sum()

48176

Subset - focus on the data collected in '2020-01' and '2020-02' 


In [15]:
case_month_boolean_mask = df_cdc['case_month'].apply(lambda x: x in ['2020-01', '2020-02'])
df_cdc_subset = df_cdc[case_month_boolean_mask]
df_cdc_subset = df_cdc_subset[['res_county','age_group','sex','race']]

In [17]:
df_cdc.loc[:, ['res_county','age_group','sex','race']].head()

,res_county,age_group,sex,race
0,ONEIDA,18 to 49 years,Male,NaN
1,ONEIDA,18 to 49 years,Male,NaN
2,ONEIDA,18 to 49 years,Male,NaN
3,ONEIDA,18 to 49 years,Male,NaN
4,ONEIDA,18 to 49 years,Male,NaN


#### Data preprocessing - handling the missing value

In [18]:
(df_cdc_subset == 'Unknown').sum()

res_county     0
age_group      0
sex            0
race          97
dtype: int64

In [19]:
df_cdc_subset[(df_cdc_subset == 'Unknown')] = None 
# just put None to this location

In [20]:
(df_cdc_subset == "Unknown").sum()

res_county    0
age_group     0
sex           0
race          0
dtype: int64

In [21]:
# after replacing na value to none
df_cdc_subset = df_cdc_subset.replace({"Unknown": None})

In [22]:
df_cdc_subset.isna().sum()

res_county     205
age_group      472
sex            612
race          1341
dtype: int64

- What is the percentage of data with one or more missing elements?
 

In [23]:
df_cdc_subset.isna().mean()

res_county    0.129911
age_group     0.299113
sex           0.387833
race          0.849810
dtype: float64

- What is the missing proportion of the race column?

In [24]:
df_cdc_subset['race'].isna().mean()

0.8498098859315589

In [25]:
df_cdc_subset.isna().mean()

res_county    0.129911
age_group     0.299113
sex           0.387833
race          0.849810
dtype: float64

In [26]:
# Remove samples with at least one missing value
df_cdc_subset_removed=df_cdc_subset.dropna().reset_index(drop=True)
df_cdc_subset_removed.head()

,res_county,age_group,sex,race
0,ONEIDA,18 to 49 years,Female,White
1,ONEIDA,18 to 49 years,Female,White
2,ONEIDA,18 to 49 years,Female,White
3,ONEIDA,18 to 49 years,Female,White
4,ONEIDA,18 to 49 years,Female,White


## Which sex is more likely to get COVID-19 in NY?

In [27]:
df_cdc_subset_removed['sex'].value_counts() 

Female    137
Male      100
Name: sex, dtype: int64

In [28]:
df_cdc_subset_removed['sex'].head()

0    Female
1    Female
2    Female
3    Female
4    Female
Name: sex, dtype: object

In [29]:
from scipy.stats import chisquare
chisquare(df_cdc_subset_removed['sex'].value_counts())

Power_divergenceResult(statistic=5.776371308016878, pvalue=0.016243041612191065)

In [30]:
df_cdc_subset_removed.head()

,res_county,age_group,sex,race
0,ONEIDA,18 to 49 years,Female,White
1,ONEIDA,18 to 49 years,Female,White
2,ONEIDA,18 to 49 years,Female,White
3,ONEIDA,18 to 49 years,Female,White
4,ONEIDA,18 to 49 years,Female,White


County level analysis

In [31]:
df_cdc_subset_removed['res_county'].unique()

array(['ONEIDA', 'BRONX', 'NEW YORK', 'QUEENS', 'KINGS'], dtype=object)

In [32]:
len(df_cdc_subset_removed['res_county'].unique()) 

5

Count the number of subjects for each county in NY

In [33]:
print(df_cdc_subset_removed.groupby('res_county').count())

            age_group  sex  race
res_county                      
BRONX              47   47    47
KINGS              66   66    66
NEW YORK           66   66    66
ONEIDA             31   31    31
QUEENS             27   27    27


Compute the proportion of Male subjects for each county in NY.

In [34]:
df_cdc_prop=df_cdc_subset_removed.groupby('res_county')['sex'].apply(lambda x: sum(x =='Male')/len(x))
df_cdc_prop

res_county
BRONX       0.000000
KINGS       0.469697
NEW YORK    0.606061
ONEIDA      0.419355
QUEENS      0.592593
Name: sex, dtype: float64

the number of counties with more female cases than males.

In [35]:
n_female_county=(df_cdc_prop < 0.5).sum()
print(f'Number of NY counties with more female cases than males: {n_female_county}')

Number of NY counties with more female cases than males: 3
